# MedX Engine, Radiography and Tomography Processing
---

## 0.0. Install Libraries 

In [ ]:
import Radiography_Library as RadLib
RadLib.Install_Libraries()

---

## 0.1. Attenuation Coefficient Bisection Algorithm

### 0.1.1. Fixed Energy Step

In [ ]:
import Radiography_Library as RadLib

threads = 10

directory = ''

root_filename = '.root'
outputcsv_name = '.csv'

root_structure = ['Transportation', 'Ratio', 'Mass_Attenuation'] # tree name, branch 1, branch 2

energies = [100000, 101000, 1000] # initial energy, final energy, energy step (in eV)

tolerance = 8

RadLib.BisectionFixedEnergyStep(directory, threads, root_filename, outputcsv_name, root_structure, energies, tolerance)

### 0.1.2. Energies from NIST CSV

In [ ]:
import Radiography_Library as RadLib

threads = 10

directory = ''

root_filename = ".root"
outputcsv_name = '.csv'

root_structure = ['Transportation', 'Ratio', 'Mass_Attenuation'] # tree name, branch 1, branch 2

input_csv = 'CdTe_nist.csv'

tolerance = 8

RadLib.BisectionEnergiesNIST(directory, threads, root_filename, outputcsv_name, root_structure, input_csv, tolerance)

### 0.1.3. Plot Attenuation Results

In [ ]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT/'

DATA = \
[
{"CSV": "CadTel_map.csv", "X": "Energy", "Y": "AtCoefficient", "LABEL": "CadTel", "MARKER": "o", "MARKERSIZE": 1, "COLOR": "red", "ALPHA": 0.5},
{"CSV": "CdTe_nist.csv",  "X": "Energy", "Y": "AtCoeff"      , "LABEL": "NIST",   "MARKER": "o", "MARKERSIZE": 1, "COLOR": "red", "ALPHA": 0.5},
]

title   = r"Element ($$)"
x_label = r"Energy ($keV$)"
y_label = r"Mass Attenuation Coefficient ($cm^2/g$)"

X_axis_log = True
Y_axis_log = True

Figure_Text = None

Font_Size_Normal = 16
Font_Size_Large  = 20

save_as = None  # '.png'

RadLib.Plot_Att_Coeff(directory, DATA, title, x_label, y_label, X_axis_log, Y_axis_log, Figure_Text, Font_Size_Normal, Font_Size_Large, save_as)

---

## 1.1. Run Radiography

In [ ]:
import Radiography_Library as RadLib

threads  = 9
energy   = 80      # keV
sim_time = 10       # min
iteration_time = 0 # min

spectra_mode        = '80kvp' # 'mono (0)' or '80kvp (1) or '140kvp (2)'
detector_parameters = {'nColumns': 1, 'nRows': 1}
gun_parameters      = {'X': 0, 'Y': 0, 'gaussX': 'true', 'SpanX': 230, 'SpanY': 240}

RadLib.RunRadiography(threads, energy, sim_time, iteration_time, spectra_mode, detector_parameters, gun_parameters, alarm=True)

## 1.2. Run DEXA

In [ ]:
import Radiography_Library as RadLib

threads  = 10
sim_time = 15      # min
iteration_time = 00 # min

spectra_mode = 'poly' # 'mono (0)' or 'poly (1)'

RadLib.RunDEXA(threads, sim_time, iteration_time, spectra_mode, detector_parameters=None, gun_parameters=None, alarm=True)

## 1.3. Merge Root Files

In [ ]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT/'
directory = '/Users/miguelcomett/GEANT4/G4_MedX'

starts_with = 'Rad_40'
starts_with = 'Rad_80'
starts_with = 'Poly_140kvp'
starts_with = 'Poly_80kvp'

output_name = starts_with

# trim_coords = (-200, 200, -200, 200)  # x_min, x_max, y_min, y_max. (slower method)
trim_coords = None

RadLib.Merge_Roots_HADD(directory, starts_with, output_name, trim_coords) 

## 1.4. Get ROOT Summary Data & Histograms 

In [ ]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT/'
directory = '/Users/miguelcomett/GEANT4/G4_MedX/'

root_file = 'Rad_40'
root_file = 'Rad_80'
root_file = 'Poly_80kvp.root'
root_file = 'Poly_140kvp.root'

hits_tree    = 'Hits';               hits_branches    = ['x_ax', 'y_ax']
summary_tree = 'Run Summary';        summary_branches = ['Number_of_Photons', 'Sample_Mass_kg', 'EDep_Value_TeV', 'Radiation_Dose_uSv']
spectra_tree = 'Energy Spectra keV'; spectra_branches = ['Energies', 'Counts']

RadLib.Summary_Data(directory, root_file, hits_tree, hits_branches, summary_tree, summary_branches, spectra_tree, spectra_branches)

range_x = [-250, 250, 1000] # range_min, range_max, bins (mm) 
range_y = [-250, 250, 1000] # range_min, range_max, bins (mm)
range_spectra = [0, 150, 300] # range_min, range_max, bins (keV)

RadLib.XY_1D_Histogram(directory, root_file, hits_tree, hits_branches, spectra_tree, spectra_branches, range_x, range_y, range_spectra)

---

## 2. Root File to Heatmap

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/GEANT4/G4_MedX/'
directory = 'BUILD/ROOT/'

root_file = 'Rad_40.root'
root_file = 'Rad_80.root'
root_file = 'Poly_80kvp.root'
root_file = 'Poly_140kvp.root'

tree_name = "Hits"; x_branch  = "x_ax"; y_branch  = "y_ax"
size = [230, 240, 0, 0] # size_x, size_y, x_shift, y_shift, 
pixel_size = 1 # mm

heatmap_raw, xlim, ylim = RadLib.Root_to_Heatmap(directory, root_file, tree_name, x_branch, y_branch, size, pixel_size, progress_bar=True)
heatmap = RadLib.Logarithmic_Transform(heatmap_raw)
RadLib.Plot_Heatmap(heatmap, save_as='')
# RadLib.Plot_Plotly(heatmap, xlim, ylim) # show interactive plot

---

## 3. DEXA: Tissue Segmentation

### Calculate Heatmaps

In [ ]:
import Radiography_Library as RadLib

directory = 'BUILD/ROOT/'
directory = '/Users/miguelcomett/GEANT4/G4_MedX/'

rootnames = ["Rad_40.root", "Rad_80.root"]
rootnames = ['Poly_80kvp.root', 'Poly_140kvp.root']

tree_name = "Hits"; x_branch  = "x_ax"; y_branch  = "y_ax"

size = [240, 240, 0, 0]

pixel_size = 0.5

low_energy_img, xlim, ylim  = RadLib.Root_to_Heatmap(directory, rootnames[0], tree_name, x_branch, y_branch, size, pixel_size, progress_bar=True)
high_energy_img, xlim, ylim = RadLib.Root_to_Heatmap(directory, rootnames[1], tree_name, x_branch, y_branch, size, pixel_size, progress_bar=True)
low_energy_img  = RadLib.Logarithmic_Transform(low_energy_img)
high_energy_img = RadLib.Logarithmic_Transform(high_energy_img)

### Save heatmaps as CSV

In [2]:
import Radiography_Library as RadLib

read_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

RadLib.Save_Heatmap_to_CSV(low_energy_img,  read_folder, '40kev_Projection2')
RadLib.Save_Heatmap_to_CSV(high_energy_img, read_folder, '80kev_Projection2')

### Read heatmap's CSV

In [4]:
import Radiography_Library as RadLib

read_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

low_energy_img  = RadLib.Read_Heatmap_from_CSV(read_folder, '40kev_Projection')
high_energy_img = RadLib.Read_Heatmap_from_CSV(read_folder, '80kev_Projection')

### Perform Segmentation

In [ ]:
import Radiography_Library as RadLib

sigma = 0
wn = 0

save_as = ['', '', '', '', '', '', '', ''] # Low Energy, High Energy, Bone [SLS], Tissue [SLS], Bone [SSH], Tissue [SSH], Bone [ACNR], Tissue [ACNR]

SLS_Bone, SLS_Tissue, SSH_Bone, SSH_Tissue, ACNR_Bone, ACNR_Tissue = RadLib.IsolateTissues(low_energy_img, high_energy_img, sigma, sigma, wn, '', save_as, save_all='')

---

## 4. Bone Mineral Density (BMD)

In [ ]:
import Radiography_Library as RadLib

thickness_bone, thickness_tissue = RadLib.BMO(SLS_Bone, SLS_Tissue)
RadLib.Plot_Heatmap(heatmap, '')
RadLib.Plot_Plotly(thickness_bone, xlim, ylim)

---

## 5.1. Calculate Interactive CNR

#### Trim Image

In [ ]:
from PIL import Image; import matplotlib.pyplot as plt

directory = 'RESULTS/'
image = 'ssh' + '.png'
image = Image.open(directory + image)
image = image.convert('L')

print(image.size)
width = image.width; height = image.height

trim = 200
# image = image.crop((trim, trim, width - trim, height - trim)) # left, top, right, bottom
# image = image.crop((8410, trim, width - 60, height - trim))

# plt.imshow(image, cmap='gray'); plt.axis('off'); plt.show()

#### Launch Interactive CNR

In [ ]:
%matplotlib widget 
%matplotlib tk
import Radiography_Library as RadLib

RadLib.Interactive_CNR(image)

## 5.2. Calculate Fixed CNR 

In [ ]:
import Radiography_Library as RadLib

image_path = "RESULTS/" + "a" + ".png"
save_as = ''

shftx_s = 0.0 # shift x-coordinate signal box
shfty_s = 0.0 
shftx_b = 200.0 # shift x-coordinate background box
shfty_b = 0.0

coords_signal  = [1200 + shftx_s, 1000 + shfty_s, 1800 + shftx_s, 1800 + shfty_s] # x1, y1, x2, y2
coords_bckgrnd = [2100 + shftx_b, 1000 + shfty_b, 2300 + shftx_b, 1800 + shfty_b] # x1, y1, x2, y2

RadLib.Fixed_CNR(image_path, save_as, coords_signal, coords_bckgrnd)

---

## 6.1. Denoise with Skimage.Denoise_Bilateral

In [ ]:
import Radiography_Library as RadLib

# load array
path = SSH_Bone
isArray = True

# or load image
if isArray == False:
    directory = 'RESULTS/'
    path = directory + 'a' + '.png'

sigma_color = 0.05
sigma_spatial = 20

Denoised_Image = RadLib.Denoise_EdgeDetection(path, isArray)

## 6.2. Denoise by Fourier Transform

In [ ]:
import Radiography_Library as RadLib

array = heatmap
isHann = False

alpha = 1

save_as = ''
isCrossSection = False # plot crosss-section

fft_image = RadLib.Denoise(array, isHann, alpha, save_as, isCrossSection)

---

## 7. Save Plotly Heatmap

In [ ]:
import Radiography_Library as RadLib

# array = htmp_array
# array = Denoised_Image
array = ACNR_Bone
array = low_energy_img

title   = r"$ \large{ \text{Thorax Radiography Projection(40 keV)} } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

width  = 800
height = 800

save_as = ''

RadLib.Plotly_Heatmap_1(array, xlim, ylim, title, x_label, y_label, width, height, save_as)

#### Plot with annotation and rectangles

In [ ]:
import Radiography_Library as RadLib

array = heatmap

title   = r"$ \large{ \text{Low energy projection (40 keV), 100M Photons} } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

sqr_1_coords = [10, 10, -10, -10]
sqr_2_coords = [10, 10, -10, -10]

annotation = 'CNR = ' 

width  = 700
height = 700

save_as = ''

RadLib.Plotly_Heatmap_2(array, xlim, ylim, title, x_label, y_label, sqr_1_coords, sqr_2_coords, annotation, width, height, save_as)

---

# 8. CT Scan

## 8.1 Run CT loop

In [ ]:
import Radiography_Library as RadLib

threads = 10
starts_with = 'CT'

angles = [341, 360]
slices = [-240, 240, 8] # negative_y, positive y, step. in mm

RadLib.CT_Loop(threads, starts_with, angles, slices, alarm=False)

## 8.2. CT Summary

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/GEANT4/G4_MedX/CT/Roots'

summary_tree = 'Run Summary';         summary_branches = ['Number_of_Photons', 'EDep_Value_TeV', 'Radiation_Dose_uSv']
spectra_tree = 'Energy Spectra keV';  spectra_branches = ['Energies', 'Counts']
spectra_tree = None

RadLib.CT_Summary_Data(directory, summary_tree, summary_branches, spectra_tree, spectra_branches)

## 8.3. Calculate projections at every angle from roots and save to CSV

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/GEANT4/G4_MedX/CT/Roots/'
csv_folder = '/Users/miguelcomett/GEANT4/G4_MedX/CT/Raw_CSV3/'

filename = 'Aang'
degrees = [0, 1, 1] # start, end, step
root_structure = ["Hits", "x_ax", "y_ax"] # tree_name, x_branch, y_branch

dimensions = [240, 240, 0, 0] # sizeX, sizeY, shiftX, shiftY. (in mm)
pixel_size = 2 # mm

raw_heatmap = RadLib.Calculate_Projections(directory, filename, degrees, root_structure, dimensions, pixel_size, csv_folder)

## 8.4. Load projections from CSV and perfrom CT reconstruction

In [ ]:
import Radiography_Library as RadLib

csv_read  = '/Users/miguelcomett/GEANT4/G4_MedX/CT/Raw_CSV2/'
csv_write = '/Users/miguelcomett/GEANT4/G4_MedX/CT/Slices/'

sigma = 0.55

degrees = [0, 359, 1] # start, end, step

slices_in  = [-240, 240, 8] # initial, final, step (in mm)
slices_out = [0, 60, 1]     # initial, final, step (index)
 
heatmap_matrix, sinogram_matrix, slices_matrix = RadLib.RadonReconstruction(csv_read, csv_write, degrees, slices_in, slices_out, sigma)

In [ ]:
RadLib.Plotly_CT(heatmap_matrix, sinogram_matrix, slices_matrix, slice=30)

In [ ]:
RadLib.MatPlotLib_CT(heatmap_matrix, sinogram_matrix, slices_matrix, step=6)

## 8.5. Export to DICOM

In [ ]:
import Radiography_Library as RadLib

csv_slices = '/Users/miguelcomett/GEANT4/G4_MedX/CT/Slices/'

mu_water = 0.26748 # 40kev
mu_water = 0.18443 # 80kev
air_tolerance = -950

HU_images = RadLib.CoefficientstoHU(csv_slices, mu_water, air_tolerance)


directory = '/Users/miguelcomett/GEANT4/G4_MedX/CT/DICOMs'
size_y = 480 # height of sample in mm
compressed = False

RadLib.Export_to_Dicom(HU_images, size_y, directory, compressed)

In [ ]:
RadLib.Plot_Heatmap(HU_images[0], save_as='')